# First fermionic DMRG

Created 03/01/2025

Objectives:
* Construct the 1D fermionic spt states from the Wang & Gu paper, first pass. Work out the case with trivial projective representation first.
* Not sure of anyway to test the states...? Apart from running the SPT classification on them.

# Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
from itertools import product

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.site import ClockSite, FermionSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel

In [5]:
def mod_4_to_bit(x):    
    possible_mod_4_values = list(range(4))
    assert x in possible_mod_4_values

    x0 = x % 2
    x1 = ((x - x0) // 2) % 2

    return (x0, x1)

In [6]:
def bit_to_mod_4(bit_pair):
    x0, x1 = bit_pair

    possible_mod_2_values = list(range(2))
    assert x0 in possible_mod_2_values
    assert x1 in possible_mod_2_values

    return x0 + 2*x1

In [7]:
def mod_4_to_bit_addition(x, y):
    (x0, x1) = mod_4_to_bit(x)
    (y0, y1) = mod_4_to_bit(y)

    out0 = (x0 + y0) % 2
    out1 = (x1 + y1) % 2

    out = bit_to_mod_4((out0, out1))

    return out

In [8]:
np.array([
    [
        mod_4_to_bit_addition(x, y) for x in range(4)
    ]
    for y in range(4)
])

array([[0, 1, 2, 3],
       [1, 0, 3, 2],
       [2, 3, 0, 1],
       [3, 2, 1, 0]])

In [9]:
def get_n1_func(n1_01, n1_10):
    l = [0, n1_01, n1_10, (n1_01 + n1_10)%2]
    
    def f(g1, g2):
        i = mod_4_to_bit_addition(g1, g2)
        return l[i]

    return f

In [10]:
n1_pair_to_fermion_operators_dict = {
    (0, 0): "Id",
    (0, 1): "C",
    (1, 0): "Cd",
    (1, 1): "N"
}

In [11]:
def get_op_list(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    left_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_left, g_in), n1_func(g_left, g_out))
    ]

    out.append((left_fermion_op_string, 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))

    right_fermion_op_string = n1_pair_to_fermion_operators_dict[
        (n1_func(g_in, g_right), n1_func(g_out, g_right))
    ]

    out.append((right_fermion_op_string, 1, 1))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [12]:
class ClusterIsingFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                self.add_multi_coupling(-1/4, op_list)

In [13]:
M = ClusterIsingFermion({'L': 20})

In [14]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [15]:
models = [
    ClusterIsingFermion({'L': 20, 'n1': p}) for p in n1_pairs
]

In [16]:
type(models[0])

__main__.ClusterIsingFermion

In [17]:
models

In [18]:
mpo = models[0].calc_H_MPO()

In [19]:
mpo.get_W(0)

<npc.Array shape=(2, 6, 4, 4) labels=['wL', 'wR', 'p', 'p*']>

In [20]:
mpo.get_W(0).to_ndarray()

array([[[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        

In [21]:
mpo.get_W(1)

<npc.Array shape=(6, 6, 2, 2) labels=['wL', 'wR', 'p', 'p*']>

In [22]:
mpo.get_W(1).to_ndarray()

array([[[[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0.,

In [23]:
mpo.get_W(2).to_ndarray()

array([[[[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  1.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  1.  ]],

        [[ 1.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  1.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  1.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  1.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ]],

        [[ 0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.

In [15]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20)
    for m in models
]

In [16]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [17]:
for psi in psis:
    psi.canonical_form()

In [18]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [19]:
psis[0]

In [20]:
psis[0].get_B(0)

<npc.Array shape=(1, 4, 1) labels=['vL', 'p', 'vR']>

In [21]:
energies

[0.0, 0.0, 0.0, 0.0]

## Non-trivial proj rep

In [24]:
pauli_proj_rep_phases = np.array([
    [1, 1, 1, 1],
    [1, 1, -1, -1],
    [1, -1, 1, -1],
    [1, -1, -1, 1]
])

In [25]:
def get_proj_rep_term_phase(group_quad):
    g_left, g_in, g_out, g_right = group_quad

    g1 = mod_4_to_bit_addition(g_left, g_in)
    g2 = mod_4_to_bit_addition(g_in, g_right)
    g3 = mod_4_to_bit_addition(g_left, g_out)
    g4 = mod_4_to_bit_addition(g_out, g_right)

    numerator_phase = pauli_proj_rep_phases[g1, g2]
    denominator_phase = pauli_proj_rep_phases[g3, g4]

    return numerator_phase/denominator_phase

In [187]:
class ClusterFermion(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list(group_quad, n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [188]:
M = ClusterIsingFermion({'L': 20})

In [189]:
n1_pairs = [
    (0, 0),
    (0, 1),
    (1, 0),
    (1, 1)
]

In [190]:
models = [
    [ClusterFermion({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [191]:
n1_pair_to_fermion_operators_dict

{(0, 0): 'Id', (0, 1): 'C', (1, 0): 'Cd', (1, 1): 'N'}

In [192]:
models

[[<__main__.ClusterFermion at 0x177bdb290>,

In [193]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [194]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [199]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
False
False
False
True
False
False
False


In [201]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 0
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [204]:
out

[[('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Cd', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('N', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('C', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_0_0', 0, 0),
  ('Id', 0, 1),
  ('map_2_2',

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [211]:
out = list()

n1_01, n1_10 = 1, 0
n1_func = get_n1_func(n1_01, n1_10)

non_trivial_proj_rep = False

g_l = 1
g_r = 0

group_pairs = product(range(4), repeat=2)

for group_pair in group_pairs:
    op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
    out.append(op_list)

In [224]:
out

[[('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_0_2', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_0_3', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_0', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_1', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('C', 0, 1),
  ('map_1_2', 1, 0),
  ('Cd', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Id', 0, 1),
  ('map_1_3', 1, 0),
  ('N', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_0', 1, 0),
  ('Id', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('Cd', 0, 1),
  ('map_2_1', 1, 0),
  ('C', 1, 1),
  ('map_0_0', 2, 0)],
 [('map_1_1', 0, 0),
  ('N', 0, 1),
  ('map_2_2', 

In [226]:
[
    (l[1][0], l[2][0], l[3][0])
    for l in out
]

[('N', 'map_0_0', 'Id'),
 ('Cd', 'map_0_1', 'C'),
 ('N', 'map_0_2', 'Id'),
 ('Cd', 'map_0_3', 'C'),
 ('C', 'map_1_0', 'Cd'),
 ('Id', 'map_1_1', 'N'),
 ('C', 'map_1_2', 'Cd'),
 ('Id', 'map_1_3', 'N'),
 ('N', 'map_2_0', 'Id'),
 ('Cd', 'map_2_1', 'C'),
 ('N', 'map_2_2', 'Id'),
 ('Cd', 'map_2_3', 'C'),
 ('C', 'map_3_0', 'Cd'),
 ('Id', 'map_3_1', 'N'),
 ('C', 'map_3_2', 'Cd'),
 ('Id', 'map_3_3', 'N')]

In [228]:
left_fermion_operators = [l[1][0] for l in out]
right_fermion_operators = [l[3][0] for l in out]

In [229]:
[left_fermion_operators[4*i:4*i+4] for i in range(4)]

[['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id'],
 ['N', 'Cd', 'N', 'Cd'],
 ['C', 'Id', 'C', 'Id']]

In [230]:
[right_fermion_operators[4*i:4*i+4] for i in range(4)]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Check hermiticity

In [216]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)
            
            g_l = model_params.get('g_l', 0)
            g_r = model_params.get('g_r', 0)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_pairs = product(range(4), repeat=2)

            for group_pair in group_pairs:
                op_list = get_op_list((g_l, *group_pair, g_r), n1_func)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [217]:
models = [
    [
        [ClusterFermionHermiticityTest({'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r}) for p in n1_pairs]
        for g_l in range(4)
    ]
    for g_r in range(4)
]

In [220]:
mpos = [[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in models]

In [221]:
[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in mpos]

[[[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]],
 [[True, False, False, False],
  [True, False, False, False],
  [True, False, False, False],
  [True, False, False, False]]]

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

### Break down into further subcases

In [279]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2)

In [280]:
models = list()

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    m = ClusterFermionHermiticityTest(
                        {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                    )
                    
                    models[-1][-1][-1][-1].append(m)

In [281]:
models[0][0][0][0][0]

In [282]:
mpos = [[[[[m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [283]:
is_hermitian = np.array([[[[[m.is_hermitian() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in mpos])

In [284]:
is_hermitian.shape

(4, 4, 4, 4, 4)

In [302]:
is_hermitian[1]

array([[[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]]],


       [[[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ True,  True, False, False],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[ True,  True, False, False],
         [ 

So breakdown in hermiticity is a function of $n_1$, $g_{in}$ and $g_{out}$, independent of $g_l$ and $g_r$.

So hermitian only if $g_{in}$ and $g_{out}$ are equal... i.e. it's the creation and annihilation terms causing problems.

In [310]:
n1_func = get_n1_func(0, 1)

g_l = 0
g_r = 0
g_in = 0
g_out = 2

In [321]:
get_op_list((g_l, 3, 3, g_r), n1_func)

[('map_0_0', 0, 0),
 ('N', 0, 1),
 ('map_3_3', 1, 0),
 ('N', 1, 1),
 ('map_0_0', 2, 0)]

In [322]:
op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)

In [323]:
op_list_1

[('map_0_0', 0, 0),
 ('C', 0, 1),
 ('map_0_2', 1, 0),
 ('C', 1, 1),
 ('map_0_0', 2, 0)]

In [324]:
op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)

In [325]:
op_list_2

[('map_0_0', 0, 0),
 ('Cd', 0, 1),
 ('map_2_0', 1, 0),
 ('Cd', 1, 1),
 ('map_0_0', 2, 0)]

Extract terms?

In [326]:
terms = models[1][0][0][2][0].all_coupling_terms()

In [327]:
terms

In [328]:
term_list = terms.to_TermList()

In [330]:
tl = term_list.terms

In [331]:
len(tl)

36

In [332]:
tl[0], tl[18]

([('map_0_0', 0), ('Cd JW', 1), ('map_2_0 JW', 2), ('Cd', 3), ('map_0_0', 4)],
 [('map_0_0', 0), ('C JW', 1), ('map_0_2 JW', 2), ('C', 3), ('map_0_0', 4)])

### Repeat but add hermitian conjugate terms

In [333]:
class ClusterFermionHermiticityTest(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    # Would it be easier to subclass ClockSite?
    def init_sites(self, model_params):
        spin = ClockSite(4, conserve=None)
        for i in range(4):
            for j in range(4):
                X = np.zeros((4,4))
                X[j, i] = 1
                op_name = f"map_{i}_{j}"
                spin.add_op(op_name, X)
        ferm = FermionSite(conserve=None)
        sites = [spin, ferm]
        return [spin, ferm], ['s', 'f']

    def init_terms(self, model_params):
        # Read off model parameters
        n1_01, n1_10 = model_params.get('n1', (0,0))
        n1_func = get_n1_func(n1_01, n1_10)

        g_l = model_params.get('g_l', 0)
        g_r = model_params.get('g_r', 0)
        g_in = model_params.get('g_in', 0)
        g_out = model_params.get('g_out', 0)

        op_list_1 = get_op_list((g_l, g_in, g_out, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_1, plus_hc=True)
        
        op_list_2 = get_op_list((g_l, g_out, g_in, g_r), n1_func)
        self.add_multi_coupling((-1/4), op_list_2, plus_hc=True)

In [339]:
models = list()
none_count = 0

for p in n1_pairs:
    models.append(list())
    for g_l in range(4):
        models[-1].append(list())
        for g_r in range(4):
            models[-1][-1].append(list())
            for g_in in range(4):
                models[-1][-1][-1].append(list())   
                for g_out in range(4):
                    try:
                        m = ClusterFermionHermiticityTest(
                            {'L': 20, 'n1': p, 'g_l': g_l, 'g_r': g_r, 'g_in': g_in, 'g_out': g_out}
                        )
                    except ValueError:
                        m = None
                        none_count += 1
                    
                    models[-1][-1][-1][-1].append(m)

In [340]:
none_count

384

$384 = 128*3 \Rightarrow 1024/384 = 8/3$

Which ones are breaking...

In [341]:
models[0][0][0][0][0]

In [342]:
mpos = [[[[[None if m is None else m.calc_H_MPO() for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models]

In [343]:
none_array = np.array([[[[[m is None for m in l1] for l1 in l2] for l2 in l3] for l3 in l4] for l4 in models])

Looks like the cases I'm interested in are exactly those missing...!

In [349]:
none_array[1]

array([[[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]]],


       [[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True, False, False],
         [ True,  True, False, False]],

        [[False, False,  True,  True],
         [F

In [223]:
FermionSite().need_JW_string

{'C', 'Cd', 'JW'}

In [208]:
fermion_operators = [l[1][0] for l in out]

In [210]:
[
    fermion_operators[:4],
    fermion_operators[4:8],
    fermion_operators[8:12],
    fermion_operators[12:16]
]

[['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N'],
 ['Id', 'C', 'Id', 'C'],
 ['Cd', 'N', 'Cd', 'N']]

In [ ]:
models[0][1].c

In [195]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [1]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())
    
    for psi, m in zip(l1, l2):
        # Commenting out to supress error message clutter in notebook.
        """
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()

        outpsis[-1].append(psi_out)
        energies[-1].append(e)
        """

NameError: name 'psis' is not defined

In [197]:
energies

[[-18.000000000000036,
  -8.196875831824364,
  -8.514815527402932,
  -8.97384686708272],
 [-18.00000000000004,
  -8.628884646999314,
  -8.279348259530973,
  -8.999999999999979]]

## ZXZ case

In [105]:
def get_xzx_op_list(g_quad):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [136]:
class Cluster(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            sites = [spin,]
            return [spin,], ['s',]

        def init_terms(self, model_params):
            # Read off model parameters
            
            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_xzx_op_list(group_quad)
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1
                self.add_multi_coupling((-1/4)*phase, op_list)

In [137]:
models = [
    Cluster({'L': 20, 'non_trivial_proj_rep': b})
    for b in [True, False]
]

In [138]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [139]:
psis = [
    MPS.from_lat_product_state(m.lat, [['0',],]*20) for m in models
]

In [140]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [141]:
for psi in psis:
    psi.canonical_form()

In [142]:
outpsis = list()
energies = list()

for psi, m in zip(psis, models):
    eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
    e, psi_out = eng.run()

    outpsis.append(psi_out)
    energies.append(e)

/Users/kierancooney/.pyenv/versions/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [143]:
energies

[-17.999999999999996, -18.000000000000046]

In [144]:
models

[<__main__.Cluster at 0x169fe90d0>, <__main__.Cluster at 0x168990690>]

In [152]:
m = models[0]

In [153]:
m.test_sanity()

In [154]:
mpo = m.calc_H_MPO()

In [155]:
mpo.is_hermitian()

True

In [156]:
m = models[1]

In [157]:
m.test_sanity()

In [158]:
mpo = m.calc_H_MPO()

In [159]:
mpo.is_hermitian()

True

In [160]:
psi = outpsis[1]

In [163]:
psi.get_B(0).to_ndarray()

array([[[1.],
        [0.],
        [0.],
        [0.]]])

In [164]:
psi.get_B(1).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])

In [165]:
psi.get_B(10).to_ndarray()

array([[[-0.5],
        [-0.5],
        [-0.5],
        [-0.5]]])

# Update hamiltonian

In [18]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [19]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                if n1_func(g_left, g_right) == 1:
                    factor = 2
                else:
                    factor = 1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [23]:
models = [
    [ClusterFermion1({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [25]:
models

[[<__main__.ClusterFermion1 at 0x7855c3d72210>,

In [26]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [27]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [28]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
True
True
True
True
True
True
True


In [29]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [30]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()
    
        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [31]:
energies

[[-17.999999999999982,
  -17.999999999999975,
  -17.999999999999964,
  -17.99999999999994],
 [-18.000000000000032,
  -18.000000000000004,
  -17.999999999999975,
  -17.99999999999997]]

Try without "factor"

In [32]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                factor=1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [33]:
models = [
    [ClusterFermion1({'L': 20, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [34]:
models

[[<__main__.ClusterFermion1 at 0x7855bd747390>,

In [35]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*20) for m in l]
    for l in models
]

In [36]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [37]:
for l in models:
    for m in l:
        mpo = m.calc_H_MPO()
        print(mpo.is_hermitian())

True
True
True
True
True
True
True
True


In [38]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [39]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
        e, psi_out = eng.run()
    
        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [40]:
energies

[[-17.999999999999982,
  -12.342466171618899,
  -12.34246617161892,
  -12.3424661716189],
 [-18.000000000000032,
  -12.342052318754007,
  -12.342146563149196,
  -12.341908845734233]]

## Check correlation lengths
Should be 0.

In [26]:
def get_op_list_1(g_quad, n1_func):
    out = list()
    g_left, g_in, g_out, g_right = g_quad

    out.append((f'map_{g_left}_{g_left}', 0, 0))

    if n1_func(g_in, g_right):
        out.append(('Cd', 1, 1))
    if n1_func(g_left, g_in):
        out.append(('Cd', 0, 1))
    
    out.append((f'map_{g_in}_{g_out}', 1, 0))
    
    if n1_func(g_left, g_out):
        out.append(('C', 0, 1))
    if n1_func(g_out, g_right):
        out.append(('C', 1, 1))

    out.append((f'map_{g_right}_{g_right}', 2, 0))

    return out

In [27]:
class ClusterFermion1(CouplingMPOModel):
        default_lattice = "Chain"
        force_default_lattice = True

        # Would it be easier to subclass ClockSite?
        def init_sites(self, model_params):
            spin = ClockSite(4, conserve=None)
            for i in range(4):
                for j in range(4):
                    X = np.zeros((4,4))
                    X[j, i] = 1
                    op_name = f"map_{i}_{j}"
                    spin.add_op(op_name, X)
            ferm = FermionSite(conserve=None)
            sites = [spin, ferm]
            return [spin, ferm], ['s', 'f']

        def init_terms(self, model_params):
            # Read off model parameters
            n1_01, n1_10 = model_params.get('n1', (0,0))
            n1_func = get_n1_func(n1_01, n1_10)

            non_trivial_proj_rep = model_params.get('non_trivial_proj_rep', False)

            group_quads = product(range(4), repeat=4)

            for group_quad in group_quads:
                op_list = get_op_list_1(group_quad, n1_func)
                
                if non_trivial_proj_rep:
                    phase = get_proj_rep_term_phase(group_quad)
                else:
                    phase = 1

                g_left, *_, g_right = group_quad
                if n1_func(g_left, g_right) == 1:
                    factor = 2
                else:
                    factor = 1

                self.add_multi_coupling((-1/4)*phase*factor, op_list)

In [60]:
models = [
    [ClusterFermion1({'L': 50, 'n1': p, 'non_trivial_proj_rep': b}) for p in n1_pairs]
    for b in [True, False]
]

In [61]:
models

[[<__main__.ClusterFermion1 at 0x73732f250a50>,

In [63]:
psis = [
    [MPS.from_lat_product_state(m.lat, [['0', 'empty'],]*50) for m in l]
    for l in models
]

In [64]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [65]:
for l in psis:
    for psi in l:
        psi.canonical_form()

In [67]:
outpsis = list()
energies = list()

for l1, l2 in zip(psis, models):
    outpsis.append(list())
    energies.append(list())

        
    for psi, m in zip(l1, l2):
        try:
            eng = dmrg.TwoSiteDMRGEngine(psi, m, dmrg_params)
            e, psi_out = eng.run()
        except:
            e, psi_out = None, None

        outpsis[-1].append(psi_out)
        energies[-1].append(e)

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/python3.11/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv_p11/lib/pytho

In [68]:
energies

[[-48.00000000000056, -47.99999999999989, -47.999999999999645, None],
 [-48.000000000000185,
  -47.99999999999984,
  -47.99999999999985,
  -47.999999999999865]]

In [69]:
len(outpsis[0][0].sites)

100

In [70]:
outpsis[0][0].get_SR(10)

array([0.5, 0.5, 0.5, 0.5])

In [71]:
outpsis[0][0].get_SR(9)

array([0.5, 0.5, 0.5, 0.5])

In [72]:
outpsis[0][0].get_SR(11)

array([0.5, 0.5, 0.5, 0.5])

In [76]:
boson_expectations = np.zeros((2, 2, 4, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                op,
                list(range(0, 100, 2))
            )
        )

        boson_expectations[i,j,k] = data

    except:
        boson_expectations[i,j,k] = np.nan

/tmp/ipykernel_12272/3510999881.py:21: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_expectations[i,j,k] = data


In [77]:
boson_expectations[0]

array([[[ 0.00000000e+00,  6.66133815e-16,  1.19695920e-16,
          2.01227923e-16,  8.25728375e-16,  1.78329573e-15,
         -1.16053001e-15, -6.55725474e-16, -9.43689571e-16,
         -6.92154667e-16, -8.36136715e-16, -1.75554016e-15,
          5.89805982e-16,  1.02695630e-15,  7.77156117e-16,
         -5.74627151e-16,  8.75601674e-16, -4.32293090e-15,
          1.24900090e-15, -3.68455266e-15, -1.80931659e-15,
          1.21864324e-15,  2.33146835e-15, -1.35308431e-16,
          1.28369537e-16, -9.12464548e-16,  2.16840434e-18,
          5.58580959e-16,  4.23272528e-16, -6.04334291e-16,
         -1.04430353e-15,  2.31759056e-15, -2.77555756e-17,
         -1.95503336e-15, -1.41553436e-15,  1.09287579e-16,
          1.74860126e-15,  3.09127723e-15, -8.95117314e-16,
         -2.17534324e-15, -3.33066907e-16, -9.05525654e-16,
         -2.71657696e-15,  1.31665512e-15,  7.28583860e-17,
          5.20417043e-17, -1.29757316e-15, -1.47798440e-15,
          4.16333634e-15,  0.00000000e+0

In [78]:
boson_expectations[1]

array([[[ 1.00000000e+00, -1.78190795e-14, -1.07136522e-14,
         -1.31145095e-15, -6.60582700e-15,  2.13717932e-15,
         -2.83106871e-14, -4.45823933e-15, -5.68989300e-15,
          2.15799600e-14, -1.61676228e-14,  1.74860126e-14,
         -1.91513472e-14, -4.86069518e-15,  1.08246745e-14,
         -1.39055434e-14,  3.02813330e-14, -1.17683641e-14,
          1.20459198e-14,  1.60149671e-14,  6.80011603e-16,
         -2.14828155e-14,  2.27734498e-14, -1.04014020e-14,
          2.76167977e-14, -2.17326157e-14,  2.14828155e-14,
         -6.85562718e-15, -1.83499049e-14, -2.22773189e-14,
          1.31700206e-14, -2.96984659e-14,  1.91235916e-14,
          2.14550600e-14,  2.81302759e-14,  2.74225087e-14,
         -2.20934382e-14, -8.82627305e-15,  3.41809914e-14,
         -1.65562009e-14, -3.27515792e-15,  1.32394096e-14,
          2.21767049e-14, -3.55826479e-14,  1.27259314e-14,
          2.61735078e-14,  3.34454686e-15,  2.09554596e-15,
          3.30638295e-15,  1.00000000e+0

In [80]:
fermion_number_expectations = np.zeros((2, 4, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .expectation_value(
                'N',
                list(range(1, 100, 2))
            )
        )

        fermion_number_expectations[j,k] = data

    except:
        fermion_number_expectations[j,k] = np.nan

In [81]:
fermion_number_expectations

array([[[0.00000000e+00, 3.01375446e-56, 1.00000000e+00, 1.00000000e+00,
         1.00000000e+00, 2.62626836e-31, 1.00000000e+00, 1.00000000e+00,
         1.08230361e-31, 4.49142978e-33, 2.89228035e-32, 1.00000000e+00,
         5.51992560e-33, 1.00000000e+00, 1.16447982e-31, 3.25843542e-29,
         1.00000000e+00, 1.46036760e-33, 8.03581152e-32, 1.00000000e+00,
         1.00000000e+00, 3.08243493e-32, 1.00000000e+00, 2.18479170e-32,
         1.40895260e-31, 7.83943620e-32, 2.81670154e-33, 1.00000000e+00,
         1.19649879e-31, 1.12591474e-32, 3.71022446e-32, 2.64253162e-32,
         2.99992830e-32, 1.00000000e+00, 1.00000000e+00, 3.96313786e-31,
         1.00000000e+00, 4.94528204e-32, 1.00000000e+00, 6.47982854e-32,
         2.15064247e-32, 2.32647044e-32, 5.44654487e-31, 2.27821783e-32,
         1.55787800e-32, 3.34399118e-31, 3.72176003e-33, 1.69611837e-32,
         1.40833975e-28, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00,
         

In [85]:
boson_correlations = np.zeros((2, 2, 4, 50, 50))

indices = product(
    enumerate(['X', 'Z']),
    range(2),
    range(4)
)

for (i, op), j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                op,
                op,
                list(range(0, 100, 2)),
                list(range(0, 100, 2)),
            )
        )
    
        boson_correlations[i,j,k] = data

    except:
        boson_correlations[i,j,k] = np.nan

/tmp/ipykernel_12272/290460682.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  boson_correlations[i,j,k] = data


In [88]:
boson_correlations[..., 1, -2]

array([[[ 6.40949485e-31,  2.50000000e-01,  0.00000000e+00,
                     nan],
        [ 1.00000000e+00,  2.50000000e-01,  1.81990196e-29,
          2.50000000e-01]],

       [[ 2.30051561e-28,  1.66533454e-16,  0.00000000e+00,
                     nan],
        [ 2.27413808e-30,  8.32667268e-17, -1.94059783e-28,
          2.41473508e-15]]])

In [91]:
fermion_correlations = np.zeros((2, 4, 50, 50))

indices = product(
    range(2),
    range(4)
)

for j, k in indices:
    try:
        psi = outpsis[j][k]
        
        data = (
            psi
            .correlation_function(
                'N',
                'N',
                list(range(1, 101, 2)),
                list(range(1, 101, 2)),
            )
        )
    
        fermion_correlations[j,k] = data

    except:
        fermion_correlations[j,k] = np.nan

In [92]:
fermion_correlations[..., 1, -2]

array([[4.24439020e-84, 0.00000000e+00, 1.00000000e+00,            nan],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.49507942e-56]])

In [93]:
reduced_boson_correlations = (
    boson_correlations -
    boson_expectations[...,np.newaxis]*boson_expectations[...,np.newaxis,:]
)

In [97]:
reduced_boson_correlations[...,1,-2]

array([[[-2.13238963e-30, -5.82867088e-16,  0.00000000e+00,
                     nan],
        [ 3.77475828e-15, -9.71445147e-16, -8.68805048e-44,
         -6.66133815e-16]],

       [[ 2.88968262e-28, -2.50000000e-01, -3.74939946e-33,
                     nan],
        [ 6.86384585e-31, -2.50000000e-01, -1.94063532e-28,
         -2.50000000e-01]]])

In [95]:
reduced_fermion_correlations = (
    fermion_correlations -
    fermion_number_expectations[...,np.newaxis]*fermion_number_expectations[...,np.newaxis,:]
)

In [98]:
reduced_fermion_correlations[...,1,-2]

array([[ 4.93770553e-98,  0.00000000e+00, -6.43929354e-15,
                    nan],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -3.62227163e-71]])